In [296]:
import os, json,unstructured_client
from unstructured_client.models import operations, shared
from unstructured_client.models import operations, shared
from unstructured_client.models.errors import SDKError
from pathlib import Path
from typing import List, Tuple
from dotenv import load_dotenv
import PyPDF2
from PyPDF2 import PdfReader
import cohere
co = cohere.ClientV2(api_key=os.getenv('cohere_api_key'))
# Load environment variables from .env file
load_dotenv()
un_client = unstructured_client.UnstructuredClient(
    api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
)
def get_elements_from_file(file_path:str,strategy:shared.Strategy=shared.Strategy.FAST):
    with open(file_path, "rb") as f:          # pass bytes, not a file handle
        file_bytes = f.read()
            # print first 10 bytes
    filename=os.path.basename(file_path)
    req = operations.PartitionRequest(
        partition_parameters=shared.PartitionParameters(
            strategy=strategy,
            files=shared.Files(
                content=file_bytes,
                file_name=filename,
                
            ),
            
            languages=["heb", "eng"],
        )
    )

    

    try:  
        res = un_client.general.partition(request=req)
        elements = res.elements                          # list of Element objects
        
        return elements
    except SDKError as e:                                # friendlier error handling
        print(f"{e.status_code}: {e.message}")

def list_all_files(
    start: str | Path,
    exts: Tuple[str, ...] = (".pdf", ".doc", ".docx"),
) -> List[str]:
    """
    Return a list of full (absolute) paths for every *PDF, DOC, or DOCX* file
    inside *start* and all its sub-directories.

    Parameters
    ----------
    start : str | Path
        The root folder to walk. Tilde (~) is expanded and the path is
        resolved so the result is always absolute.
    exts : tuple[str, ...], optional
        File-name suffixes to include (case-insensitive). Default picks
        '.pdf', '.doc', and '.docx'.

    Raises
    ------
    FileNotFoundError
        If *start* does not exist or is not a directory.
    """
    root = Path(start).expanduser().resolve()

    if not root.is_dir():
        raise FileNotFoundError(f"{root} is not an existing directory")

    # rglob('*') walks recursively; filter by suffix (case-insensitive)
    return [
        str(p)
        for p in root.rglob("*")
        if p.is_file() and p.suffix.lower() in exts
    ]

def check_for_scanned_file(file_path:str)->bool:
    
    """
    Check if the file is scanned or not.
    
    Args:
        file (File): The File object to be checked.
        
    Returns:
        bool: True if the file is scanned, False otherwise.
    """
    pdf = PyPDF2.PdfReader(file_path)
    pages= pdf.pages
    # Check if the file has any images
    if len(pages[0].extract_text()) == 0:
        # If the first page has no text, assume it's a scanned file
        return True
    else:
        # If the first page has text, assume it's not a scanned file
        return False

In [252]:
pathes=list_all_files("docs")

In [266]:
for path in pathes:
    if not path.endswith(".pdf"):
        continue


    if (check_for_scanned_file(path)):
        print(f"File {os.path.basename(path) } is scanned")
    else:
        print(f"File {os.path.basename(path) } is not scanned")
        # elements = get_elements_from_file(path)
        # Print the elements

    

File ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf is not scanned
File GaLore- Memory-Efficient LLM Training by Gradient Low-Rank Projection.pdf is not scanned
File 2403.06634.pdf is not scanned
File RMP - Bonded Manifolds - Rev B_scanned for TF.pdf is scanned
File RMP - Bonded Manufolds - Rev A.pdf is scanned
File RMF - bonded manifolds - Rev C.pdf is scanned
File RMF- Bonded Manifolds -Rev G_scanned for TF.pdf is scanned
File RMP - Transducer modifications - Rev A.pdf is scanned
File RMF- DPT & DIPT Rev  C.pdf is scanned
File VCR#21-009 Kingstec.pdf is scanned
File VCRֳ#21-005 Kingstec.pdf is scanned
File RMP - Closed Stopcock & MRVLS - Rev B_scanned for TF.pdf is scanned
File General RMF - Closed Stopcock & MRVLS.pdf is scanned
File RMF - closed STP - Rev E.pdf is scanned
File RMF- Closed Stopcock - Rev H_scanned for TF.pdf is scanned
File RMP - Close STPs & MRVLS- Rev A.pdf is scanned


In [268]:
unstructerd_files=[]
for i in range(5):
    if pathes[i].endswith(".pdf"):
        if check_for_scanned_file(pathes[i]):
            strategy=shared.Strategy.VLM
        else:
            strategy=shared.Strategy.FAST
    elif pathes[i].endswith(".doc") or pathes[i].endswith(".docx"):
        strategy=shared.Strategy.FAST
    el=get_elements_from_file(pathes[i],strategy)
    unstructerd_files.append(el)


INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1

In [228]:
for file in unstructerd_files:
    print(file[0]['metadata']['filename'])
    


RMF- Bonded Manifolds -Rev G_scanned for TF.pdf
RMP - Bonded Manifolds.doc
סקר סיכונים הדבקת פלטות.docx


In [275]:
import pandas as pd
cols=['type','element_id','text','metadata','filename','page_number','text_as_html','languages','filetype','partitioner_type']
metadata_cols=['filename','page_number','text_as_html','languages','filetype','partitioner_type']
df=pd.DataFrame(columns=cols)
# df=pd.read_csv('unstructured_df.csv')
df

,type,element_id,text,metadata,filename,page_number,text_as_html,languages,filetype,partitioner_type


In [270]:

def add_file_to_df(file: list, df: pd.DataFrame):
    
    for element in file:
        # Check if the element already exists in the DataFrame
        if df[df['element_id'] == str(element)].empty:
            # If it doesn't exist, create a new row
            pass
        else:
            # If it exists, skip to the next element
            continue
        # Create a dictionary for the row
        row_dict = {
            'type': element['type'],
            'element_id': str(element),
            'text': element['text'],
            'metadata': element['metadata'] 
        }
        
        # Extract each metadata column if it exists
        for meta_col in metadata_cols:
            if meta_col in element['metadata']:
                row_dict[meta_col] = element['metadata'][meta_col]
            else:
                row_dict[meta_col] = None  # Set to None if metadata field doesn't exist
                
        # Using concat instead of append (which is deprecated)
        df = pd.concat([df, pd.DataFrame([row_dict])], ignore_index=True)
    return df
# Iterate over the list of files and add each to the DataFrame





In [280]:
for file in unstructerd_files:
    df = add_file_to_df(file, df)
    # for el in file:
        # print(el)


In [283]:
#count how many rows have an empty text
unempty_files = df[df['text'].str.strip() != '']
unempty_files

,type,element_id,text,metadata,filename,page_number,text_as_html,languages,filetype,partitioner_type
0,Title,"{'type': 'Title', 'element_id': '1b50c69fd4e27...",ArtPrompt: ASCII Art-based Jailbreak Attacks a...,"{'languages': ['heb', 'eng'], 'page_number': 1...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1,None,"[heb, eng]",application/pdf,None
1,UncategorizedText,"{'type': 'UncategorizedText', 'element_id': '6...",♣,"{'languages': ['heb', 'eng'], 'page_number': 1...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1,None,"[heb, eng]",application/pdf,None
2,NarrativeText,"{'type': 'NarrativeText', 'element_id': '8708f...",WARNING: This paper contains model outputs tha...,"{'languages': ['heb', 'eng'], 'page_number': 1...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1,None,"[heb, eng]",application/pdf,None
3,UncategorizedText,"{'type': 'UncategorizedText', 'element_id': '4...",♣‡,"{'languages': ['heb', 'eng'], 'page_number': 1...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1,None,"[heb, eng]",application/pdf,None
4,UncategorizedText,"{'type': 'UncategorizedText', 'element_id': '2...",♣‡,"{'languages': ['heb', 'eng'], 'page_number': 1...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1,None,"[heb, eng]",application/pdf,None
...,...,...,...,...,...,...,...,...,...,...
2846,ListItem,"{'type': 'ListItem', 'element_id': 'c45eb708ec...",Document Revision Control:,"{'category_depth': 0, 'filename': 'Risk Manage...",Risk Management Plan - Bonded Manifolds.doc,None,None,[eng],application/msword,None
2847,Table,"{'type': 'Table', 'element_id': '666d7a3208420...",Revision History Tracking: Date: Revision 0- f...,{'emphasized_text_contents': ['Revision Histor...,Risk Management Plan - Bonded Manifolds.doc,None,<table><tr><td>Revision History Tracking:</td>...,[eng],application/msword,None
2848,ListItem,"{'type': 'ListItem', 'element_id': 'beebf563ba...",Review and approval:,"{'category_depth': 0, 'filename': 'Risk Manage...",Risk Management Plan - Bonded Manifolds.doc,None,None,[eng],application/msword,None
2849,Table,"{'type': 'Table', 'element_id': 'a2f1fc0cec9c9...",Written by: Name Date Signature QA Eng. - - -,"{'emphasized_text_contents': ['Written by:', '...",Risk Management Plan - Bonded Manifolds.doc,None,<table><tr><td>Written by:</td><td>Name</td><t...,"[eng, deu]",application/msword,None


In [ ]:
# df.to_csv('unstructured_df.csv', index=False)

In [279]:
len(df)

2851

In [300]:
import chromadb

client=chromadb.PersistentClient('path="chroma_db")')
collection_name='first_collection'
collection=client.get_or_create_collection(name=collection_name)
from chromadb.utils import embedding_functions
embed_model = "embed-v4.0" 
embedding_function=embedding_functions.CohereEmbeddingFunction(api_key=os.getenv("COHERE_API_KEY"), model_name=embed_model)
chunk_size=20
chunk_overlap=3

0        True
1        True
2        True
3        True
4        True
        ...  
2123    False
2124    False
2125    False
2126    False
2127    False
Name: element_id, Length: 2128, dtype: bool

In [255]:
df

,type,element_id,text,metadata,filename,page_number,text_as_html,language,filetype,partition_type
0,UncategorizedText,6b37173e9c5180e77bfe620773f2ef81,NaN,"{'category_depth': 0, 'page_number': 1, 'text_...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<div class=""Page"" data-page-number=""1"" id=""7d1...",NaN,application/pdf,NaN
1,Title,f04ab8375d52fc9874f380be7a6e2fd1,ArtPrompt: ASCII Art-based Jailbreak Attacks a...,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<h1 class=""Title"" id=""b2f3acf3a4a0427da98fe18d...",NaN,application/pdf,NaN
2,NarrativeText,4820ee097d1de1dce5cd53b612fa2c3e,⚠ WARNING: This paper contains model outputs t...,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<p class=""NarrativeText"" id=""7d3efa9742344716a...",NaN,application/pdf,NaN
3,UncategorizedText,46a00f49ac4707e94aef48456d972513,NaN,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<section class=""Section"" id=""dbde263fa0474785a...",NaN,application/pdf,NaN
4,NarrativeText,b9749709852d13efd2aca7020ae3c607,Fengqing Jiang*‡ Zhangchen Xu*‡ Luyao Ni...,"{'category_depth': 2, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<p class=""NarrativeText"" id=""8f1e0c539ff5483f8...",NaN,application/pdf,NaN
...,...,...,...,...,...,...,...,...,...,...
2123,UncategorizedText,"{'type': 'UncategorizedText', 'element_id': 'a...",החלטות שהתקבלו:,"{'category_depth': 0, 'languages': ['heb'], 'f...",סקר סיכונים הדבקת פלטות.docx,NaN,NaN,NaN,application/vnd.openxmlformats-officedocument....,NaN
2124,ListItem,"{'type': 'ListItem', 'element_id': '591084b0d1...",פתיחת פעילות חדשה- להגדרת חיי מדף של פלטה מוד...,"{'category_depth': 0, 'languages': ['heb'], 'f...",סקר סיכונים הדבקת פלטות.docx,NaN,NaN,NaN,application/vnd.openxmlformats-officedocument....,NaN
2125,ListItem,"{'type': 'ListItem', 'element_id': 'ce8d16ae9c...",הפעילות תמוקם בספרייתActivities Non Project.,"{'category_depth': 0, 'languages': ['heb'], 'f...",סקר סיכונים הדבקת פלטות.docx,NaN,NaN,NaN,application/vnd.openxmlformats-officedocument....,NaN
2126,ListItem,"{'type': 'ListItem', 'element_id': '9576414801...","לצורך הפעילות יוגדרו שני מקט""ים מייצגים, בעלי ...","{'category_depth': 0, 'languages': ['heb'], 'f...",סקר סיכונים הדבקת פלטות.docx,NaN,NaN,NaN,application/vnd.openxmlformats-officedocument....,NaN


In [ ]:
# def add_element_to_collection(element, collection):
    

In [301]:
def text_to_embedding(text):
    res=co.embed(
        model=embed_model,
        input_type="search_document",
        embedding_types=["float"],
        texts=[text],
    ).embeddings.float[0]
    return res
    

In [314]:
row=df.iloc[0]
# print(row['text'])
# text_to_embedding(row['text'])
row['metadata']['languages']=str(row['metadata']['languages'])
collection.add(
    documents=[row['filename']],
    metadatas=[row['metadata']],
    ids=[row['element_id']],
    embeddings=[text_to_embedding(row['text'])],
)

INFO: HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"


In [316]:
query="what is the name of the file?"
results=collection.query(
    query_embeddings=[text_to_embedding(query)],
    n_results=1,
    include=["documents", "metadatas", "distances"],
)
results

INFO: HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"


{'ids': [["{'type': 'Title', 'element_id': '1b50c69fd4e278e6fc164601259be612', 'text': 'ArtPrompt: ASCII Art-based Jailbreak Attacks against Aligned LLMs', 'metadata': {'languages': ['heb', 'eng'], 'page_number': 1, 'filename': 'ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf', 'filetype': 'application/pdf'}}"]],
 'embeddings': None,
 'documents': [['ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf']],
 'uris': None,
 'included': ['documents', 'metadatas', 'distances'],
 'data': None,
 'metadatas': [[{'page_number': 1,
    'languages': "['heb', 'eng']",
    'filetype': 'application/pdf',
    'filename': 'ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf'}]],
 'distances': [[1.593124508857727]]}